# Background

Before starting with data sketching algorithms for genomics, I wanted to give a quick primer on some basic concepts that will be used frequently later. This will also save us from repeating a lot of code in the other notebooks!

This notebook will cover:

* k-mers
 - what are they
 - how to generate them from a sequence
 - canonical k-mers

* genomic data streams
 
* more coming...

Alternatively, skip straight to another notebook:

* Sketching algorithms
    -   [3.1 Set similarity with MinHash](../notebooks/r3.1.Set-similarity-with-MinHash.ipynb)
    -   [3.2 Set membership with Bloom filter](../notebooks/r3.2.Set-membership-with-Bloom-filters.ipynb)
    -   [3.3 Element frequency with Count-Min sketch](../notebooks/r3.3.Element-frequency-with-Count-Min-sketch.ipynb)
    -   [3.4 Set cardinality with HyperLogLog](../notebooks/r3.4.Set-cardinality-with-HyperLogLog.ipynb)
* Workflows for genomics
    -   [4.1 Overview](../notebooks/r4.1.Workflows-for-genomics.ipynb)
    -   [4.2 Sample QC](r4.2.Sample-QC.ipynb)
    -   [4.3 Resistome profiling](r4.3.Resistome-profiling.ipynb)
    -   [4.4 Outbreak surveillance](r4.4.Outbreak-surveillance.ipynb)
    -   [4.5 Data mining](r4.5.Data-mining.ipynb)
    
***

## K-mers

### The basics

From [Wikipedia](https://en.wikipedia.org/wiki/K-mer):

> k-mers refer to all the possible subsequences (of length k) from a read obtained through DNA Sequencing

So, given a sequence that is 10 nucleotides long:

```
ACTGACTGAC
```

We can *decompose* it to the constituent k-mers. Here, we will use k=7:

```
ACTGACT
 CTGACTG
  TGACTGA
   GACTGAC
```

You'll notice that for any sequence, the number of k-mers you can get is equal to the sequence length (L) minus the k-mer size, plus 1:

```
number of k-mers = L - k + 1
```

So, for our sequence that is 10 nucleotides long, we got 4 k-mers that were 7 nucleotides long:

```
4 = 10 - 7 + 1
```

But why use k-mers at all?

> We use k-mers a lot in bioinformatics because they are an exact length, as opposed to reads or genomes which are typically variable in length. Computers prefer things to be exact lengths; making processing quicker for fixed lengths compared to variable lengths.

Now let's use some Python to decompose sequences to k-mers...

***

* Start with defining a sequence:

In [ ]:
sequence = "ACTGACTGAC"

* To decompose this sequence into the constituent k-mers of length 7, start with calculating the number of k-mers we can get:

In [ ]:
sequenceLength = len(sequence)
kmerSize = 7
numberKmers = sequenceLength - kmerSize + 1
print("Number of {}-mers possible from {}\t=\t{}\n" .format(kmerSize, sequence, numberKmers))

* By sliding a window that is 7 nucleotides long across the sequence one nucleotide at a time, we can determine each k-mer:

> We use the number of possible k-mers from the last step so that we know when to stop sliding the window.

In [ ]:
for i in range(numberKmers):
    print(sequence[i:i+kmerSize])

* Let's now combine the last few bits of code into a function that we can re-use:

In [ ]:
"""
getKmers
- takes a sequence and a k-mer size
- decomposes the sequence to k-mers
- returns a list of k-mers
"""
def getKmers(seq, k):
    # a list to store the k-mers
    kmers = []
    
    # calculate the number of possible k-mers
    numKmers = len(seq) - k + 1
    
    # slide a window along the sequence and store the k-mer
    for i in range(numKmers):
        kmer = seq[i:i + k]
        kmers.append(kmer)
    return kmers

* Test the function out:

In [ ]:
print(getKmers(sequence, 5))
print(getKmers(sequence, 6))
print(getKmers(sequence, 7))

### Canonical k-mers

As DNA is usually double stranded, we need to take into account the **reverse complement** of each k-mer that we observe as we may have sequenced either strand and we want to consider both.

> An observation of a k-mer (say ACTG), means that its reverse complement (CAGT) also exists in the genome we have sequenced, as it is just the same bit of genome read but has been read in the opposite direction.

To make sure that we are considering both k-mers, we use something called the **canonical k-mer**, which is the lexicographically smaller of the k-mer and its reverse complement. This just means the k-mer which comes first when sorted alphabetically.

To get the reverse complement, we will make our lives easier and use the excellent [screed](https://github.com/dib-lab/screed/) library. This library contains many great utilies for working with short read data. We'll be using this library for some other stuff later too.

* Reverse complement a k-mer:

In [ ]:
import screed

kmer = "TACTGACTG"
rcKmer = screed.rc(kmer)
print(kmer)
print(rcKmer)

* The lexicgraphically smaller of the two is the canonical k-mer:

In [ ]:
canonical=""
if kmer <= rcKmer:
    canonical = kmer
else:
    canonical = rcKmer
print(canonical)

* Let's update our getKmers function to take into account canonical k-mers:

In [ ]:
"""
getKmers
- takes a sequence and a k-mer size
- decomposes the sequence to canonical k-mers
- returns a list of k-mers
"""
def getKmers(seq, k):
    # a list to store the k-mers
    kmers = []
    
    # calculate the number of possible k-mers
    numKmers = len(seq) - k + 1
    
    # slide a window along the sequence and get each k-mer
    for i in range(numKmers):
        kmer = seq[i:i + k]
        
        # get reverse complement
        rcKmer = screed.rc(kmer)
        
        # store the canonical k-mer
        if kmer <= rcKmer:
            kmers.append(kmer)
        else:
            kmers.append(rcKmer)
    return kmers

* Test the updated function:

In [ ]:
print(getKmers(sequence, 5))
print(getKmers(sequence, 6))
print(getKmers(sequence, 7))

### K-mer decomposition of genomes

Now that we can decompose a sequence into the constituent canonical k-mers, let's try this for some genomes. We'll use screed again, this time to handle file parsing:

In [ ]:
"""
genome2kmers
- takes a file and a k-mer size
- decomposes the genome to canonical k-mers
- returns a list of k-mers
"""

def genome2kmers(filename, k):
    # a list to store the k-mers
    kmers = []

    # open the file and read the sequences
    for record in screed.open(filename):
        sequence = record.sequence
        
        # get the canonical k-mers and add them to the list
        kmers += getKmers(sequence, k)

    return kmers

> NOTE: these functions in the notebook aren't very efficient but are a good way of illustrating the concepts. Nor do they feature any checking (e.g. for non-ACTG nucleotides). I just thought I'd mention that now before we move on.


* Test it out:

In [ ]:
genomeKmers = genome2kmers('../data/GCF_000025565.1_ASM2556v1_genomic.fna.gz', 31)
print(genomeKmers[:10])

## Genomic data streams

### The basics

In the review paper, I made a big deal about the fact that sketching is great for dealing with data streams; the point was that we can use sketching algorithms to summarise data as it's being generated (or received etc.).

A data stream is essentially just continuous data; it can come from multiple sources and needs to be processed sequenctially and incrementally. Some examples are:

* telemetry from some sort of monitoring device (e.g. heart rate monitor)

* web data (e.g. content views)

* customer logs (e.g. shop footfall)

In the case of genomics, a data stream could be:

* the download (or upload) of sequence data to a server

* the generation of sequencing data (e.g. Nanopore)

* the alignment of sequence reads against a reference


To simulate a genomic data stream for these notebooks, we can use a Python [generator](https://wiki.python.org/moin/Generators). They basically allow you to start using data before it has finished being generated. So, where you had to wait a minute in the previous cell for our **genome2kmers** function to return a list of k-mers, we can use a generator to start giving us k-mers as soon as we start processing the genome.

We'll adapt the previous function (genome2kmers) to now give us a generator. We will then use it in the other notebooks to simulate a genomic data stream.

In [ ]:
"""
streamGenome
- takes a file and a k-mer size
- decomposes the genome to canonical k-mers
- returns a Python generator that will yield the k-mers one at a time
"""
def streamGenome(filename, k):

    # open the file and read the sequences
    for record in screed.open(filename):
        sequence = record.sequence

        # calculate the number of possible k-mers
        numKmers = len(sequence) - k + 1

        # slide a window along the sequence and get each k-mer
        for i in range(numKmers):
            kmer = sequence[i:i + k]

            # get reverse complement
            rcKmer = screed.rc(kmer)

            # get the canonical k-mer
            if kmer <= rcKmer:
                yield kmer
            else:
                yield rcKmer

* Test it out by streaming the first 10 k-mers from the genome:

In [ ]:
dataStream = streamGenome('../data/GCF_000025565.1_ASM2556v1_genomic.fna.gz', 31)
for i in range(0,10):
    print(next(dataStream))

Now, let's look at some sketching algorithms:

* [3.1 Set similarity with MinHash](../notebooks/r3.1.Set-similarity-with-MinHash.ipynb)
* [3.2 Set membership with Bloom filter](../notebooks/r3.2.Set-membership-with-Bloom-filters.ipynb)
* [3.3 Element frequency with Count-Min sketch](../notebooks/r3.3.Element-frequency-with-Count-Min-sketch.ipynb)
* [3.4 Set cardinality with HyperLogLog](../notebooks/r3.4.Set-cardinality-with-HyperLogLog.ipynb)